In [ ]:
# 1. Easier Specification of Server Counts - done
# 2. Non-linear Power Consumption - done
# 3. Modeling Networking Gear - done
# 4. Stochastic Workloads - todo
# 5. Time-Dependent Variations - done
# 6. More Granular Cooling Models - no
# 7. Integrate Renewable Energy Sources - no
# 8. Power Usage Effectiveness (PUE) as Dynamic - no
# 9. Add Cost and Carbon Emission Tracking - todo
# 10. Visualization and Reporting - todo
# 11. More modular design?
# 12. Different simulation libraries like SimPy or Monte Carlo?

import random

class Load:
    """Time-varying server load model."""
    def __init__(self):
        # For now, sinusoidal load model with peak in the afternoon.
        # Load between 0 and 1, in 2-hour intervals starting at 0:00-1:59.
        self.load_by_hours = [ 0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 1.0, 1.0, 1.0, 0.8, 0.6, 0.6 ]
    def load_at_hour(self, hour):
        assert 0 <= hour < 24
        return max(0, min(self.load_by_hours[hour // 2] + random.uniform(-0.1, 0.1), 1))

load = Load()

class Server:
    def __init__(self, power_w):
        self.power_w = power_w

    def power_at_hour(self, hour):
        l = load.load_at_hour(hour)
        return self.power_w * (0.6 * l + 0.4 * l * l)

    def energy_consumption(self, start_hour, end_hour):
        # Integrate power over time based on load model.
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24 and start_hour < end_hour
        e = 0
        for hour in range(start_hour, end_hour):
            e += self.power_at_hour(hour)
        return e

class NetworkGear:
    def __init__(self, power_w):
        self.power_w = power_w

    def energy_consumption(self, start_hour, end_hour):
        # No load factor because power utilization at >10% load is ~= power utilization at 100% load.
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24 and start_hour < end_hour
        return self.power_w * (end_hour - start_hour)

class DataCenter:
    def __init__(self, servers, network_gear, cooling_power_kw, lighting_power_kw):
        self.servers = servers
        self.network_gear = network_gear
        self.cooling_power_kw = cooling_power_kw
        self.lighting_power_kw = lighting_power_kw

    def total_energy_consumption(self, hours):
        self.compute_kwh = (sum(server.energy_consumption(0, 23) for server in self.servers) / 1000)
        network_kwh = (sum(gear.energy_consumption(0, 23) for gear in self.network_gear) / 1000)
        cooling_kwh = self.cooling_power_kw * hours
        lighting_kwh = self.lighting_power_kw * hours
        self.total_energy = self.compute_kwh + network_kwh + cooling_kwh + lighting_kwh
        return self.total_energy

    def pue(self):
        self.total_energy_consumption(24)
        return self.total_energy / self.compute_kwh if self.compute_kwh > 0 else float('inf')

# Example usage
servers = (
    [Server(100) for _ in range(1000)] +
    [Server(500) for _ in range(100)] +
    [Server(1000) for _ in range(20)]
)
network_gear = (
    [NetworkGear(70) for _ in range(25)] +
    [NetworkGear(90) for _ in range(5)]
)

print(f"Total number of servers: {len(servers)}")
cooling_power_kw = 20  # Cooling system power consumption in kW
lighting_power_kw = 1  # Lighting power consumption in kW

data_center = DataCenter(servers, network_gear, cooling_power_kw, lighting_power_kw)
hours = 24  # Number of hours to calculate energy consumption for
total_energy = data_center.total_energy_consumption(hours)
pue = data_center.pue()
print(f"Total energy consumption for {hours} hours: {total_energy:.2f} kWh")
print(f"PUE: {pue:.2f}")


Total number of servers: 1120
Total energy consumption for 24 hours: 2889.12 kWh
PUE: 1.24
